In [27]:
import math

import numpy
from numpy.ma.extras import row_stack
# import numpy as np
from pgmpy.readwrite import BIFReader
import random
import copy
from collections import deque
from itertools import  product

In [28]:
#Parameters
GROUP_ID = '04'
ALGORITHM = 've'
NETWORK_NAME = 'networks/child.bif'
REPORT = '[Disease]'
EVIDENCE_LEVEL = 'Little'
EVIDENCE = 'LowerBodyO2="<5"; RUQO2=12+; CO2Report=">=7.5"; XrayReport=Asy/Patchy'

In [29]:
class Factor():
    def __init__(self, cpd = None):
        if(cpd == None):
            self.table = 0
            self.variables = {}
        else:
            self.table = {}
            self.variables = cpd.variables
            possibleCombos = []
            fixedCardinality = []
            for var in self.variables:
                fixedCardinality.append(len(cpd.name_to_no[var]))
            for x in fixedCardinality:
                l = []
                for num in range(x):
                    l.append(num)
                possibleCombos.append(l)
            possibleCombos = list(product(*possibleCombos))
            for combo in possibleCombos:
                val = cpd.values
                varVals = []
                for i in range(len(combo)):
                    val = val[combo[i]]
                    varVals.append(cpd.no_to_name[self.variables[i]][combo[i]])
                self.table[tuple(varVals)] = val


    def restrict(self, variable, value):
        variableIndex = self.variables.index(variable)
        for entry in list(self.table.keys()):
            if(entry[variableIndex] == value):
                val = self.table[entry]
                entryList = list(entry)
                entryList.pop(variableIndex)
                self.table.pop(entry)
                entry = tuple(entryList)
                self.table[entry] = val
            else:
                self.table.pop(entry)
        self.variables.remove(variable)

    def varIn(self, variable):
        if variable in self.variables:
            return True
        return False


    def sumOut(self, variable):
        variableIndex = self.variables.index(variable)
        newTable = {}
        #repeat until all values have been summed
        while(len(self.table)> 0):
            #get the first entry in the table
            entry1 = list(self.table.keys())[0]
            removeEntries = []
            valueList = []
            valueList.append(self.table[entry1])
            #look through the list for other entries to sum with
            for entry in list(self.table.keys())[1:]:
                if(sameExceptOneIndex(entry1, entry, variableIndex)):
                    removeEntries.append(entry)
                    valueList.append(self.table[entry])
            #creating the new entry (with new value and key without original variable)
            entryList = list(entry1)
            entryList.pop(variableIndex)
            self.table.pop(entry1)
            entry = tuple(entryList)
            #trying to account for floating point error
            newTable[entry] = round(math.fsum(valueList), 8)
            #removing all entries that were used to sum
            for e in removeEntries:
                self.table.pop(e)
        self.variables.remove(variable)
        self.table = newTable

    def multiply(self,other):
        resultFactor = Factor()
        # Union of variables
        resultFactor.variables = []
        for v in self.variables:
            if v not in resultFactor.variables:
                resultFactor.variables.append(v)
        for v in other.variables:
            if v not in resultFactor.variables:
                resultFactor.variables.append(v)
        domains = []
        for var in resultFactor.variables:
            values = set()
            if var in self.variables:
                i = self.variables.index(var)
                for vals in self.table:
                    values.add(vals[i])

            if var in other.variables:
                i = other.variables.index(var)
                for vals in other.table:
                    values.add(vals[i])
            domains.append(list(values))
        # Generate product
        allAssignments = []

        '''
        def build_assignments(prefix, depth):
            if depth == len(domains):
                allAssignments.append(tuple(prefix))
                return
            for val in domains[depth]:
                build_assignments(prefix + [val], depth + 1)

        build_assignments([], 0)
        '''
        allAssignments = list(product(*domains))
        # Fill resultFactor.table 
        resultFactor.table = {}

        for assignment in allAssignments:

            # Map variable --> value
            assignmentDict = {}
            for i in range(len(resultFactor.variables)):
                assignmentDict[resultFactor.variables[i]] = assignment[i]
            if self.variables:
                key1 = []
                for var in self.variables:
                    key1.append(assignmentDict[var])
                key1 = tuple(key1)
                val1 = self.table[key1]
            else:
                val1 = 1.0
            if other.variables:
                key2 = []
                for var in other.variables:
                    key2.append(assignmentDict[var])
                key2 = tuple(key2)
                val2 = other.table[key2]
            else:
                val2 = 1.0

            # Multiply
            resultFactor.table[assignment] = val1 * val2

        return resultFactor



def sameExceptOneIndex(val1, val2, index):
    for i in range(len(val1)):
        if((val1[i] != val2[i]) and (i != index)):
            return False
    return True





def variableElimination(network, queryVars, evidenceVars):
    print(queryVars, evidenceVars)
    factors = []
    #creating a factor for each cpd
    for cpd in network.cpds:
        factors.append(Factor(cpd))

    #restricting known values
    for variable in evidenceVars:
        for factor in factors:
            if(factor.varIn(variable)):
                factor.restrict(variable, evidenceVars[variable])




In [30]:
def reverseOrder(network):
    queue = deque()
    visited = set()
    order = []
    roots = network.get_roots()
    for i in roots:
        queue.append(i)
        visited.add(i)
    while queue:
        cur = queue.popleft()
        order.append(cur)
        for i in network.get_children(cur):
            if i not in visited:
                queue.append(i)
                visited.add(i)
    order.reverse()
    return order







In [31]:
def normalizeDistribution(distribution):
    #Normalizes the numbers in a distribution to sum to 1
    sum = 0
    for value in distribution:
        sum += value
    if sum != 0:
        for i in range(len(distribution)):
            distribution[i] = distribution[i] / sum
    return distribution

def normalizeVEDistribution(distribution):
    sum = 0
    answer = []
    #Normalizes the distribution created in variable elim
    for value in distribution:
        sum += distribution[value]
    if sum != 0:
        for value in distribution:
            distribution[value] = distribution[value] / sum
            answer.append(distribution[value])
    return answer

In [32]:
def gibbsSampling(Network, reportedVars, evidenceVars, numSamples, burnInLength):
    possVals = Network.states
    randomVals = {}

    #Initializing random state values
    for key in possVals.keys():
        if key in evidenceVars.keys():
            randomVals[key] = evidenceVars[key]
        else:
            randomVals[key] = random.choice(possVals[key])

    #initializing counting tables
    countingTables = []
    for var in reportedVars:
        countingTable = []
        for i in range(len(possVals[var])):
            #case where one of the reported vars is given as evidence
            if(var in evidenceVariables):
                countingTable.append('x')
            else:
                countingTable.append(0)
        countingTables.append(countingTable)

    for i in range(numSamples):
        #Need to pick a random (non evidence) node use its markov blanket
        randVar = random.choice(list(Network.nodes))
        while(randVar in evidenceVars.keys()):
            randVar = random.choice(list(Network.nodes))
        mb = getParentsAndChildren(Network, randVar)
        #Calculate probability of variable given its parents
        cpds = Network.get_cpds(randVar)
        valueDistribution = []
        for value in cpds.state_names[randVar]:
            #Calculates P(x_i|parents(X_i))
            parents = mb[0]
            valueArrIndex = cpds.name_to_no[randVar][value]
            valueArr = cpds.values[valueArrIndex]
            for parent in parents:
                parentVal = randomVals[parent]
                parentArrIndex = cpds.name_to_no[parent][parentVal]
                valueArr = valueArr[parentArrIndex]
            varVal = valueArr
            childrenProb = 1
            #Calculates/sums all child probabilities P(y_j | parents(y_j))
            for children in mb[1]:
                childCpds = Network.get_cpds(children)
                parents = Network.get_parents(children)
                childrenArrIndex = childCpds.name_to_no[children][randomVals[children]]
                childrenArr = childCpds.values[childrenArrIndex]
                for parent in parents:
                    if(parent != randVar):
                        parentVal = randomVals[parent]
                    else:
                        parentVal = value
                    parentArrIndex = childCpds.name_to_no[parent][parentVal]
                    childrenArr = childrenArr[parentArrIndex]
                #Fix for 0 probabilities causing the sampling to get stuck in a state
                if(childrenArr == 0):
                    childrenArr = 0.05
                childrenProb *= childrenArr
            valueDistribution.append(varVal*childrenProb)
        #normalized value distribution for randomly selected variable has been found
        valueDistribution = normalizeDistribution(valueDistribution)
        possibleVals = Network.states[randVar]
        chosenVal = numpy.random.choice(possibleVals, p = valueDistribution)
        #setting the variable's randomly (probability-distribution) based value
        randomVals[randVar] = chosenVal
        #counting value for reported variable if not in burn in period
        if(i >= burnInLength):
            for t in range(len(countingTables)):
                countingIndex = possVals[reportedVars[t]].index(randomVals[reportedVars[t]])
                #check for case where reported variable is also a part of evidence
                if(countingTables[t][0] != 'x'):
                    countingTables[t][countingIndex] += 1


    for table in countingTables:
        if(table[0] != 'x'):
            table = normalizeDistribution(table)
    return countingTables


def getParentsAndChildren(Network, variable):
    #Gets a sorted list of parents and children variables of a random variable
    #Used for easier access of a variable's markov blanket
    mb = Network.get_markov_blanket(variable)
    nodes = []
    parents = Network.get_parents(variable)
    children = []
    for node in mb:
        nodeParents = Network.get_parents(node)
        if variable in nodeParents:
            children.append(node)
    nodes.append(parents)
    nodes.append(children)
    return nodes

In [33]:
def createOutput(reportVariables, network, probabilityDistribution):
    #writes the file output for gibbs sampling to a text file
    fileName = GROUP_ID + '_' + ALGORITHM + '_' + NETWORK_NAME.lstrip('networks/').rstrip('.bif') + '_' + EVIDENCE_LEVEL + '.csv'
    states = network.states
    with(open(fileName, 'w') as file):
        counter = -1
        for var in reportVariables:
            file.write(var)
            counter += 1
            for state in states[var]:
                file.write(",")
                file.write(state)
            file.write("\n")
            if ALGORITHM == "gibbs":
                for i in range(len(probabilityDistribution[counter])):
                    file.write(str(probabilityDistribution[counter][i]))
                    if i != len(probabilityDistribution[counter])-1:
                        file.write(",")
                    else:
                        file.write("\n")
            else:
                file.write(str(probabilityDistribution[counter]))
                file.write("\n")


#function for sanitizing input
def removeSpace(string):
    newStr = ''
    for c in string:
        if c != ' ':
            newStr += c
    return newStr



reader = BIFReader(NETWORK_NAME)
model = reader.get_model()
reportList = REPORT.lstrip('[').rstrip(']')
reportVariables1 = reportList.split(',')
reportVariables = []
for var in reportVariables1:
    reportVariables.append(removeSpace(var))
evidenceVariables = {}
if(EVIDENCE_LEVEL != "None"):
    splitter = EVIDENCE.split(";")
    for var in splitter:
        #Edge case for variables with '=' characters in their values
        if'"' in var:
            splitter2 = var.split('"')
            evidenceVariables[splitter2[0][:-1].strip()] = splitter2[1].strip()
        else:
            splitter2 = var.split("=")
            evidenceVariables[splitter2[0].strip()] = splitter2[1].strip()


#runs the algorithm corresponding to the input
if ALGORITHM == "gibbs":
    probDist = gibbsSampling(model,reportVariables,evidenceVariables, 210000, 10000)
    createOutput(reportVariables, model, probDist)
elif ALGORITHM == "ve":
    for variable in reportVariables:
        probDist = variableElimination(model, variable, evidenceVariables)
        #createVEOutput(model,probDist,variable)

else:
    print("Unrecognized algorithm:", ALGORITHM)





Disease {'LowerBodyO2': '<5', 'RUQO2': '12+', 'CO2Report': '>=7.5', 'XrayReport': 'Asy/Patchy'}


In [34]:
reader = BIFReader(NETWORK_NAME)
model = reader.get_model()
cpds = model.cpds
testCpd = cpds[0]
testCpd1 = cpds[1]
test1 = Factor(cpd = testCpd)
test2 = Factor(cpd = testCpd1)
print("test1:\n", test1.variables, "\n\n", test1.table)
test1.sumOut("Sick")
print("test2:\n",test2.variables, "\n\n", test2.table)
prod = test1.multiply(test2)
print("prod:\n",prod.variables, "\n\n", prod.table)

test1:
 ['Age', 'Disease', 'Sick'] 

 {('0-3_days', 'PFC', 'yes'): np.float64(0.95), ('0-3_days', 'PFC', 'no'): np.float64(0.85), ('0-3_days', 'TGA', 'yes'): np.float64(0.8), ('0-3_days', 'TGA', 'no'): np.float64(0.7), ('0-3_days', 'Fallot', 'yes'): np.float64(0.7), ('0-3_days', 'Fallot', 'no'): np.float64(0.25), ('0-3_days', 'PAIVS', 'yes'): np.float64(0.8), ('0-3_days', 'PAIVS', 'no'): np.float64(0.8), ('0-3_days', 'TAPVD', 'yes'): np.float64(0.8), ('0-3_days', 'TAPVD', 'no'): np.float64(0.7), ('0-3_days', 'Lung', 'yes'): np.float64(0.9), ('0-3_days', 'Lung', 'no'): np.float64(0.8), ('4-10_days', 'PFC', 'yes'): np.float64(0.03), ('4-10_days', 'PFC', 'no'): np.float64(0.1), ('4-10_days', 'TGA', 'yes'): np.float64(0.15), ('4-10_days', 'TGA', 'no'): np.float64(0.2), ('4-10_days', 'Fallot', 'yes'): np.float64(0.15), ('4-10_days', 'Fallot', 'no'): np.float64(0.25), ('4-10_days', 'PAIVS', 'yes'): np.float64(0.15), ('4-10_days', 'PAIVS', 'no'): np.float64(0.15), ('4-10_days', 'TAPVD', 'yes'